### 简历项目2：使用QLoRA+AWQ处理基于LLaMA3.1-8B的电商客服机器

#### 1. 项目简介
&emsp;&emsp; 在实际电商客服场景中，客服对话通常具有指令强、回复短、上下文依赖弱、风格固定等特点，而通用大模型往往存在回复冗长、不符合业务规范、推理成本高的问题。<br>
&emsp;&emsp; 因此，本项目以电商客服单轮对话为切入点，利用真实业务数据对 LLaMA3.1-8B 进行 监督微调（SFT），使模型学习“客服式回答”的语言模式与约束。同时，通过 QloRA 技术在有限算力条件下完成高质量微调，并在此基础上进一步采用 AWQ 量化，在尽量不损失性能的前提下降低模型推理成本，从而探索一条 **小显存训练 + 低成本部署的电商大模型落地方案**。

#### 2. 数据集
&emsp;&emsp; **E_commerce_Customer_Service_dataset** 可以从魔搭社区下载，电商客户服务数据集，核心定位是为电商客服机器人的研究提供数据支持。它包含从lightinthebox网站收集的电商数据，包括通用问题的问答对以及商品信息，**样本展示**<br>
&emsp;&emsp;{"messages": [{"role": "system", "content": "你是一个专业的电商客服。严格遵守：1) 仅以‘assistant’角色回答一次；2) 不要提出新问题，不要延长对话；3) 不要生成‘user’或其他角色的内容；4) 输出只包含答案正文，无前后缀角色标签、无思考过程。"}, {"role": "user", "content": "那要怎样这也不行那也不行"}, {"role": "assistant", "content": "发邮政的哦"}]} <br>
&emsp;&emsp;{"messages": [{"role": "system", "content": "你是一个专业的电商客服。严格遵守：1) 仅以‘assistant’角色回答一次；2) 不要提出新问题，不要延长对话；3) 不要生成‘user’或其他角色的内容；4) 输出只包含答案正文，无前后缀角色标签、无思考过程。"}, {"role": "user", "content": "可以优惠价么"}, {"role": "assistant", "content": "我们因为200把起给优惠价格您可以给您105元一把哦"}]}<br>
&emsp;&emsp;{"messages": [{"role": "system", "content": "你是一个专业的电商客服。严格遵守：1) 仅以‘assistant’角色回答一次；2) 不要提出新问题，不要延长对话；3) 不要生成‘user’或其他角色的内容；4) 输出只包含答案正文，无前后缀角色标签、无思考过程。"}, {"role": "user", "content": "我去不早说发韵达能到我家那儿我就能拿到"}, {"role": "assistant", "content": "韵达不发的哦"}]}<br>

#### 2. SFT阶段
&emsp;&emsp;参考连接：https://blog.csdn.net/u013172930/article/details/147363767 <br>
&emsp;&emsp;引入 QloRA 方法，在冻结大部分参数的情况下完成高效微调，显著降低训练显存需求。QLoRA在LoRA基础上引入量化，原始模型使用4-bit量化加载（通常为NF4），训练部分仍然是LoRA的低秩参数(FP16/BF16)。所以，QLoRA = LoRA+ 4-bit量化的大模型加载。QLoRA相比于LoRA而言在相同算力资源下，可以训练更大的模型，但其实现复杂度较高，因量化计算的需要导致训练速度会下降。<br>
&emsp;&emsp;QLoRA的优化有三个核心要点：首先是定义了一种4位标准浮点数（Normal Float 4-bit，NF4）量化，基于分块的分位数量化的量化策略；其次是双重量化，包含对普通参数的一次量化和对量化常数的再一次量化，可以进一步减小缓存占用；最后是分页优化器（Page Optimizer），用来在显存过高时用一部分内存代替显存。
##### 2.1 4位标准浮点数量化
&emsp;&emsp;QLoRA 将 预训练大模型的权重 从 FP16 / BF16 量化为 4-bit，具体采用的是 NF4（NormalFloat4） 数据格式。每个参数只占 4 bit，权重在显存中以量化形式存储，计算时动态反量化到高精度参与计算。量化技术基于 Block-Wise Quantization，将权重矩阵分块量化以优化精度和效率。<br>
&emsp;&emsp; 为什么不用普通 int4？普通 int4 是均匀量化，而大模型权重分布通常近似正态分布：大量权重集中在 0 附近
极端值较少，NF4 是非均匀量化，专门为正态分布设计：在 0 附近有更高分辨率，在尾部用更粗的刻度。在同样 4-bit 条件下，NF4 比 int4 信息损失更小。<br>
&emsp;&emsp; **QLoRA存储用 4-bit，计算用高精度**，权重存储：4-bit NF4（极省显存），前向 / 反向计算：FP16 / BF16，从来不直接用 4-bit 做矩阵乘法，所以必须有 反量化（dequantization） 这一步。为什么不能直接用 4-bit 做计算？当前主流 GPU对 4-bit 浮点（NF4）几乎没有原生计算单元。
##### 2.2 LoRA技术
 &emsp;&emsp;见[简历项目1](resume_project1_DPO_Recom.ipynb) 中的LoRA介绍
##### 2.3 双重量化
&emsp;&emsp;QLoRA 引入了双重量化技术，进一步优化内存使用。第一层量化将权重量化为 4-bit，第二层量化将量化的缩放因子（scaling factors）也量化为较低精度（如 8-bit），减少存储开销。
##### 2.4 分页优化（Paged Optimizers）：
&emsp;&emsp;QLoRA 使用 NVIDIA 的统一内存技术，将优化器状态（如 Adam 的动量）分页到 CPU 内存，降低 GPU 内存需求。


#### 3. AWQ量化阶段
&emsp;&emsp;参考链接：https://zhuanlan.zhihu.com/p/697761176 https://zhuanlan.zhihu.com/p/682041025 <br>
<div align="center">
   <img src="./pics/rp2_awq_1.png" width="690" />
</div>
&emsp;&emsp;AWQ（Activation-aware Weight Quantization）量化是一种基于激活值分布(activation distribution)挑选显著权重(salient weight)进行量化的方法，其不依赖于任何反向传播或重建，因此可以很好地保持LLM在不同领域和模式上的泛化能力，而不会过拟合到校准集，属训练后量化(Post-Training Quantization, PTQ)大类。<br>

##### 3.1  Improving LLM Quantization by Preserving 1% Salient Weights
&emsp;&emsp;QLoRA作者指出，模型的权重并不同等重要，仅有0.1%~1%的小部分显著权重（salient weights）对模型输出精度影响较大。因此如果能有办法只对0.1%~1%这一小部分权重保持原来的精度(FP16)，对其他权重进行低比特量化，就可以在保持精度几乎不变的情况下，大幅降低模型内存占用，并提升推理速度。作者发现基于激活值分布挑选(based on act.列)的方式相比FP16几乎没有精度损失，是重大发现。<br>

##### 3.2 Protecting Salient Weights by Activation-aware Scaling
&emsp;&emsp;对于激活值较大的通道，其对应权重在推理中被频繁且大幅度使用，量化误差会被显著放大。AWQ 通过为这些“重要通道”引入 activation-aware scaling factor，在量化前对权重进行放大，使量化误差在整数空间中相对减小；在推理时再将该缩放抵消，从而在不引入额外计算开销的前提下，有效保护关键权重。该方法无需重新训练模型，仅依赖少量校准数据即可在低比特（如 4-bit）权重量化下保持接近全精度模型的推理性能。<br>

&emsp;&emsp;AWQ 通过引入与激活分布相关的 scaling factor 𝑆 ，在量化前对权重进行重标定，使得量化误差在输出空间中近似均衡，从而在低比特权重量化下有效保护对模型输出最敏感的权重通道。


**不引入反量化（纯推理友好）**<br>
&emsp;&emsp;和 QLoRA 完全不同，AWQ：不做反量化，不需要 FP16 权重，不参与训练，推理时：权重始终是 int4 / int8，直接整数 GEMM，极快、极省显存，AWQ 是为部署而生的。<br>
&emsp;&emsp;AWQ 不微调模型，只需要：几十到几百条样本，和真实使用场景分布一致即可。

#### 4. 面试提问
1. 描述一下这个项目 <br>
1. 描述一下数据集，数据是如何得到的，数据集做了哪些预处理 <br>
1. 如何评估最后的模型 <br>
1. lora微调的参数设置是如何 <br>
1. 量化之后的模型相比量化前性能变化是怎样的? <br>